In [1]:
import torch
import models
from sklearn.neighbors import DistanceMetric
from utils import utils
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn.functional as F
import numpy as np
import train
import myexman
import os
from dfd_utils.FaceForensicsDataset import FaceForensicsDataset
from torch.utils.data import Subset
from torch.utils.data import Dataset, DataLoader
import yaml
from dfd_utils.utils import plot_images, get_embeddings, plot_embeddings_2D, plot_embeddings_3D
import pandas as pd
from argparse import Namespace
from sklearn.neighbors import NearestNeighbors
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
plt.rcParams['figure.dpi']=300
device = torch.device('cuda')


# Compare the perfomance of the linear classifier to that of the KNN algorithm:
specifically we wanted to test the ability to detect different types of deepfake manipulations even ones not used in training
We want to evaluate these two method using 3 different encoders to create the latent space - SimCLR with the original augmentations, SimCLR with the new FF oriented augmentations and a ResNet-50 pretrained on ImageNet (no contrastive training)


## Load models:

In [3]:
original_aug_ckpt = torch.load('/srv/DeepFakeDetection/andrew_atonov_simclr_pytorch/simclr-pytorch/logs/exman-train.py/runs/000213/checkpoint-87500.pth.tar', map_location=device)
original_aug_ckpt['hparams'].deepfakes = False # don't put deepfakes in the train set
original_aug_model = models.ssl.SSLEval.load(original_aug_ckpt, device=device)

new_aug_ckpt = torch.load('/media/shirbar/DATA/Deep_Learning/Project/Azure_results/000028 - classifier new_augs pretrained/checkpoint-87500.pth.tar', map_location=device)
new_aug_ckpt['hparams'].deepfakes = False # don't put deepfakes in the train set
new_aug_ckpt['hparams'].encoder_ckpt = '/media/shirbar/My Passport/trained_models/simclr_new_augs_pretrained_only_real.pth.tar'
new_aug_model = models.ssl.SSLEval.load(new_aug_ckpt, device=device)

#new_aug_no_pre_ckpt = torch.load('/srv/DeepFakeDetection/andrew_atonov_simclr_pytorch/simclr-pytorch/logs/exman-train.py/runs/000212/checkpoint-87500.pth.tar', map_location=device)
new_aug_no_pre_ckpt = torch.load('/media/shirbar/DATA/Deep_Learning/Project/Azure_results/000004 - linear eval divided no pretrain/checkpoint-87500.pth.tar')
new_aug_no_pre_ckpt['hparams'].deepfakes = False 
new_aug_no_pre_ckpt['hparams'].encoder_ckpt = '/media/shirbar/My Passport/trained_models/simclr_new_transforms_real_epoch_11000.pth.tar'
new_aug_no_pre_model = models.ssl.SSLEval.load(new_aug_no_pre_ckpt, device=device)

new_aug_no_pre_ckpt2 = torch.load('/srv/DeepFakeDetection/andrew_atonov_simclr_pytorch/simclr-pytorch/logs/exman-train.py/runs/000212/checkpoint-87500.pth.tar', map_location=device)
new_aug_no_pre_ckpt2['hparams'].deepfakes = False 
new_aug_no_pre_model2 = models.ssl.SSLEval.load(new_aug_no_pre_ckpt2, device=device)

resnet_ckpt = torch.load('/srv/DeepFakeDetection/andrew_atonov_simclr_pytorch/simclr-pytorch/logs/exman-train.py/runs/000216/checkpoint-87500.pth.tar', map_location=device)
resnet_ckpt['hparams'].deepfakes = False # don't put deepfakes in the train set
resnet_model = models.ssl.SSLEval.load(resnet_ckpt, device=device)

original_aug_model.encoder.eval()
original_aug_model.eval()

new_aug_model.encoder.eval()
new_aug_model.eval()

new_aug_no_pre_model.encoder.eval()
new_aug_no_pre_model.eval()

resnet_model.encoder.eval()
resnet_model.eval()

device is: cuda
device is: cuda
dp
** Removing original FC layer **
** Using avgpool **
** Loading pretrained ResNet50 weights **
======> Encoder: output dim 2048 | 23.508M parameters
device is: cuda
device is: cuda
dp
** Removing original FC layer **
** Using avgpool **
** Loading pretrained ResNet50 weights **
======> Encoder: output dim 2048 | 23.508M parameters
device is: cuda
device is: cuda
dp
** Removing original FC layer **
** Using avgpool **
======> Encoder: output dim 2048 | 23.508M parameters
device is: cuda
device is: cuda
dp
** Removing original FC layer **
** Using avgpool **
======> Encoder: output dim 2048 | 23.508M parameters
device is: cuda
device is: cuda
dp
** Removing original FC layer **
** Using avgpool **
** Loading pretrained ResNet50 weights **
======> Encoder: output dim 2048 | 23.508M parameters


# Load train dataset and create/load embeddings:
creating the embeddings may take some time, you can optionally upload them from a saved file

In [4]:
# the train set images are the same for all models so we can just load the train set once:
original_aug_model.prepare_data()
train_loader, _ = original_aug_model.dataloaders()

preparing data
calling transforms from SSLeval class
importing FF
FF train dir: /media/shirbar/My Passport/FaceForensics/split_ds/train
FF test dir: /media/shirbar/My Passport/FaceForensics/split_ds/test
** Using original SimCLR transforms
Compose(
    CenterCropAndResize(proportion=0.95, size=224)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    <function SSLEval.transforms.<locals>.<lambda> at 0x7f32e852c840>
)
loading real images...
real images loaded!
real imgs len: 366831
masks len: 366831
fakes imgs len: 0
fakes masks len: 0
final imgs len: 366831
final masks len: 366831
asserting order
assertion passed!
loading real images...
real images loaded!
real imgs len: 73768
masks len: 73768
fakes imgs len: 0
fakes masks len: 0
final imgs len: 73768
final masks len: 73768
asserting order
assertion passed!
Train size: 28000
Test size: 4000


In [5]:
# Calculate embeddings:
@torch.no_grad()
def get_embeds(model, loader, div=True):
    embeds = []
    if div:
        div_embeds = []
    targets = []
    counter=0
    for image, _, target in tqdm(loader):
        image = image.to(device)
        if counter==0:
            print(f'before: {image.max()}, {image.min()}')
        embeds.append(model.encode(image))
        if div:
            image = image/255.
            if counter==0:
                print(f'after: {image.max()}, {image.min()}')
            div_embeds.append(model.encode(image))
        targets.append(target)
        counter+=1
    
    embeds = torch.vstack(embeds)
    targets = torch.hstack(targets)
    if div:
        div_embeds = torch.vstack(div_embeds)
        return embeds.cpu(), div_embeds.cpu(), targets.cpu()
    return embeds.cpu(), targets.cpu()


In [5]:
_, original_aug_train_embeds, train_targets = get_embeds(original_aug_model, train_loader)
_, new_aug_train_embeds, _ = get_embeds(new_aug_model, train_loader)
_, new_aug_no_pre_train_embeds, _ = get_embeds(new_aug_no_pre_model, train_loader)
_, new_aug_no_pre_train_embeds2, _ = get_embeds(new_aug_no_pre_model2, train_loader)
_, resnet_train_embeds,_ = get_embeds(resnet_model, train_loader)


NameError: name 'train_loader' is not defined

In [157]:
# save your embeddings if you wish:
torch.save({'embeddings': original_aug_train_embeds, 'targets':train_targets},'/media/shirbar/My Passport/trained_models/embeddings/original_augs_all_train_set_embeds_real.pt')
torch.save({'embeddings': new_aug_train_embeds, 'targets':train_targets},'/media/shirbar/My Passport/trained_models/embeddings/new_augs_all_train_set_embeds_real.pt')
torch.save({'embeddings': resnet_train_embeds, 'targets':train_targets},'/media/shirbar/My Passport/trained_models/embeddings/resnet_all_train_set_embeds_real.pt')
torch.save({'embeddings': new_aug_no_pre_train_embeds, 'targets':train_targets},'/media/shirbar/My Passport/trained_models/embeddings/new_augs_no_pre_all_train_set_embeds_real.pt')
torch.save({'embeddings': new_aug_no_pre_train_embeds2, 'targets':train_targets},'/media/shirbar/My Passport/trained_models/embeddings/new_augs_no_pre_all_train_set_embeds_real2.pt')


## alternatively you can load the embeddings we used from file:

In [6]:
org_aug_path = '/media/shirbar/My Passport/trained_models/embeddings/original_augs_all_train_set_embeds_real.pt'
new_aug_path = '/media/shirbar/My Passport/trained_models/embeddings/new_augs_all_train_set_embeds_real.pt'
new_aug_no_pre_path = '/media/shirbar/My Passport/trained_models/embeddings/new_augs_no_pre_all_train_set_embeds_real.pt'
new_aug_no_pre_path2 = '/media/shirbar/My Passport/trained_models/embeddings/new_augs_no_pre_all_train_set_embeds_real.pt'
resnet_path = '/media/shirbar/My Passport/trained_models/embeddings/resnet_all_train_set_embeds_real.pt'

chkpt = torch.load(org_aug_path)
original_aug_train_embeds, train_targets = chkpt['embeddings'], chkpt['targets']
chkpt = torch.load(new_aug_path)
new_aug_train_embeds = chkpt['embeddings']
chkpt = torch.load(new_aug_no_pre_path)
new_aug_no_pre_train_embeds = chkpt['embeddings']
chkpt = torch.load(new_aug_no_pre_path2)
new_aug_no_pre_train_embeds2 = chkpt['embeddings']
chkpt = torch.load(resnet_path)
resnet_train_embeds = chkpt['embeddings']

# Function to create a custom test loader:

In [7]:
def load_ff_ds_test(wanted_ds_size,
                    batch_size,
                    load_deepfakes=False,
                    load_face2face=False,
                    load_neural_textures=False,
                    masking_transforms=False,
                    transform=None,
                   split='test'):
    TEST_DATASET_PATH = f'/media/shirbar/My Passport/FaceForensics/split_ds/{split}'
    ff_ds_test  = FaceForensicsDataset(TEST_DATASET_PATH,
                                    transform=transform,
                                    load_deepfakes=load_deepfakes,
                                    load_face2face=load_face2face,
                                    load_neural_textures=load_neural_textures,
                                  masking_transforms=masking_transforms)
    
    ff_ds_test.equalize_real_fakes()
    #"""
    if wanted_ds_size != -1:
        test_size = int(0.8 * len(ff_ds_test))
        ts_size = len(ff_ds_test) - test_size

        test_dataset,_ = torch.utils.data.random_split(ff_ds_test, [test_size, ts_size], generator=torch.Generator().manual_seed(42))
        test_subset = Subset(test_dataset, list(range(0, wanted_ds_size)))
    else:
        test_subset = ff_ds_test
    #"""
    """
    ff_ds_test_len = len(ff_ds_test)
    wanted_ds_size = wanted_ds_size if ff_ds_test_len > wanted_ds_size else ff_ds_test_len
    random.seed(42)
    random_indices = random.sample(range(0,ff_ds_test_len), wanted_ds_size)

    test_subset = Subset(ff_ds_test, random_indices)
    """
    test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle = False)

    print('test size:', test_subset.__len__())
    return test_loader

# Model evaluation function for the linear classifier:

In [8]:
@torch.no_grad()
def old_eval_model_classification(model,data_loader,no_pre=False):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    preds = []
    target = []
    for images,_,labels in tqdm(data_loader):
        if no_pre:
            images = images.to(device).float()
        else:
            images = images.to(device)/255.#.float()
        h = model.encoder(images, out='h').float()
        pred = model(h)
        pred = F.softmax(pred,dim=1)
        pred_class = torch.argmax(pred, dim=1)
        preds.append(pred_class.cpu())
        target.append(labels)
        
    p = torch.cat(preds).numpy()
    t = torch.cat(target).numpy()
    total_correct = np.sum(p==t)
    acc = total_correct / len(data_loader.dataset) * 100
    print('\ntotal real:', np.sum(t), ' total fake:', len(data_loader.dataset) - np.sum(t))
    print('\nPREDS:\n', p,'\nTARGETS:\n',t)
    print('total correct:', total_correct, ' out of:', len(data_loader.dataset))
    print(f'\nACC: {acc}')
    return acc, p, t

@torch.no_grad()
def eval_model_classification(model,h,targets):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    h = h.to(device)
    targets = targets.to(device)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    pred = model(h.to(device))
    pred = F.softmax(pred,dim=1)
    pred_classes = torch.argmax(pred, dim=1)
    total_correct = (pred_classes==targets).sum()
    acc = total_correct / len(targets) * 100
    print('\ntotal real:', targets.sum(), ' total fake:', len(targets) - targets.sum())
    print('\nPREDS:\n', pred_classes,'\nTARGETS:\n',targets)
    print('total correct:', total_correct, ' out of:', len(targets))
    print(f'\nACC: {acc}')
    return acc, pred_classes, targets

# Functions for KNN based classifier:
after https://arxiv.org/abs/2002.10445, the semi supervised setting

In [10]:
# make sure to run the get_embeds cell a few cells up

def get_train_knns(train_embeds, k=3,p=2):
    #dist = DistanceMetric.get_metric('euclidean')
    #train_dists = dist.pairwise(train_embeds.cpu())
    #train_knns = np.take_along_axis(train_dists,
    #                                np.argpartition(train_dists + np.diag(np.ones(train_dists.shape[0]) * 10),
    #                                    k, axis=1)[:, :k], axis=1)
    #train_knns.sort(axis=1)
    
    # we want the k nearest neighbors of each vector in the train set, for that we need to first define k+1 neighbors
    # this is because the closest neighbor each vector will be itself, and we want to ignore this self distance
    nbrs = NearestNeighbors(n_neighbors=k+1, algorithm='auto',p=p).fit(train_embeds) 
    train_knns, _ = nbrs.kneighbors(train_embeds)
    train_knns = train_knns[:,1:] # remove first column as it is only zeros (each vector's dist to itself)
    nbrs.n_neighbors = k # return back to k, the test vecs won't be in the train set 
    return nbrs, train_knns

def get_test_knns(train_neighbors,test_embeds,k=3):
    #test_knns = []
    #for vec in test_embeds:
    #    test_knns.append(np.sort((((vec-train_embeds)**2).sum(axis=1)).sqrt().cpu())[:k])
    #test_knns = np.vstack(test_knns)
    assert train_neighbors.n_neighbors == k
    test_knns, _ = train_neighbors.kneighbors(test_embeds.cpu())
    return test_knns

def eval_model_knns(test_embeds, test_targets, train_neighbors, train_knns,k=3):
    test_knns = get_test_knns(train_neighbors,test_embeds,k)
    # Note: the paper didn't specify how/what was their thresholding for the KNN distances, so we 
    # selected to use the 95th quantile of the KNN distances of the training data, additional tuning on the
    # validation set will probably yield better results.
    thresh = np.quantile(train_knns,0.95) 
    # real has a label of 1 so anything below the thresh should be labelled 1:
    predictions = (test_knns.mean(axis=1)<thresh).astype(int) 
    predictions = torch.Tensor(predictions)
    total_correct = (predictions==test_targets).sum()
    acc = total_correct / len(test_knns) * 100
    return acc, predictions, test_targets, thresh



# Load test sets:

In [11]:
TEST_SIZE = 6000
TRANSFORMS = original_aug_model.trainset.dataset.dataset.transform #all classifiers have the same transforms
split='val'

dpfk_test_loader = load_ff_ds_test(wanted_ds_size=TEST_SIZE,
                    batch_size=16,
                    load_deepfakes=True,
                    load_face2face=False,
                    load_neural_textures=False,
                    masking_transforms=False,
                    transform=TRANSFORMS,
                    split=split) 
f2f_test_loader = load_ff_ds_test(wanted_ds_size=TEST_SIZE,
                    batch_size=16,
                    load_deepfakes=False,
                    load_face2face=True,
                    load_neural_textures=False,
                    masking_transforms=False,
                    transform=TRANSFORMS,
                    split=split)
nt_test_loader = load_ff_ds_test(wanted_ds_size=TEST_SIZE,
                    batch_size=16,
                    load_deepfakes=False,
                    load_face2face=False,
                    load_neural_textures=True,
                    masking_transforms=False,
                    transform=TRANSFORMS,
                    split=split)


loading real images...
real images loaded!
loading deepfakes...
deepfakes loaded!
real imgs len: 68506
masks len: 68506
fakes imgs len: 68511
fakes masks len: 68511
final imgs len: 137017
final masks len: 137017
asserting order
assertion passed!
before eq: real: 68506  fake: 68511  total: 137017
after eq: total: 137012
test size: 6000
loading real images...
real images loaded!
loading face2face...
face2face loaded!
real imgs len: 68506
masks len: 68506
fakes imgs len: 0
fakes masks len: 0
final imgs len: 68506
final masks len: 68506
asserting order
assertion passed!
before eq: real: 68506  fake: 0  total: 68506
after eq: total: 68506
test size: 6000
loading real images...
real images loaded!
loading neural textures...
neural textures loaded!
real imgs len: 68506
masks len: 68506
fakes imgs len: 0
fakes masks len: 0
final imgs len: 68506
final masks len: 68506
asserting order
assertion passed!
before eq: real: 68506  fake: 0  total: 68506
after eq: total: 68506
test size: 6000


In [57]:
batch = next(iter(train_loader))

before 255, 0before 255, 0before 255, 0
before 255, 0


before 255, 0
after transform
tensor(255, dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor(255, dtype=torch.uint8) after transformtensor(0, dtype=torch.uint8)before 255, 0

after transformafter transform
tensor(239, dtype=torch.uint8)

 tensor(0, dtype=torch.uint8)tensor(255, dtype=torch.uint8)tensor(240, dtype=torch.uint8)   tensor(239, dtype=torch.uint8) tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)  
tensor(255, dtype=torch.uint8)tensor(240, dtype=torch.uint8) after transform 
tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)

tensor(255, dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor(255, dtype=torch.uint8)before 255, 0 before 255, 0tensor(0, dtype=torch.uint8)


before 255, 0before 255, 0before 255, 0
after transform


tensor(236, dtype=torch.uint8) after transformbefore 255, 0
tensor(0, dtype=torch.uint8)
 tensor(248, dtype=torch.uint8)tensor(236, dtype=torch.uint8)

tensor(27, dtype=torch.uint8) tensor(255, dtype=torch.uint8)tensor(205, dtype=torch.uint8)
  tensor(255, dtype=torch.uint8)before 255, 0tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)
   tensor(255, dtype=torch.uint8) tensor(0, dtype=torch.uint8)tensor(205, dtype=torch.uint8)before 255, 0 tensor(0, dtype=torch.uint8)


tensor(0, dtype=torch.uint8)
after transform
tensor(255, dtype=torch.uint8) tensor(26, dtype=torch.uint8)after transform after transformtensor(255, dtype=torch.uint8)

 tensor(26, dtype=torch.uint8)tensor(255, dtype=torch.uint8)tensor(230, dtype=torch.uint8) 
 after transformtensor(0, dtype=torch.uint8)
 tensor(0, dtype=torch.uint8)tensor(255, dtype=torch.uint8)  tensor(255, dtype=torch.uint8)tensor(230, dtype=torch.uint8)tensor(0, dtype=torch.uint8)  
tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8) 
tensor(255, dtype=torch.uint8)before 255, 0 
before 255, 0before 255, 0tensor(0, dtype=torch.uint8)


before 255, 0after transform

after transformbefore 2

before 255, 0tensor(231, dtype=torch.uint8)
 tensor(0, dtype=torch.uint8) tensor(231, dtype=torch.uint8) tensor(0, dtype=torch.uint8)
before 255, 0
after transform
tensor(255, dtype=torch.uint8)after transform 
before 255, 0before 255, 0tensor(2, dtype=torch.uint8)tensor(255, dtype=torch.uint8)

  tensor(0, dtype=torch.uint8)tensor(255, dtype=torch.uint8) tensor(255, dtype=torch.uint8)  tensor(2, dtype=torch.uint8)tensor(0, dtype=torch.uint8)

before 255, 0after transformbefore 255, 0


tensor(232, dtype=torch.uint8) after transformtensor(0, dtype=torch.uint8)
 tensor(232, dtype=torch.uint8)tensor(255, dtype=torch.uint8)  tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)
after transform 
tensor(255, dtype=torch.uint8) tensor(236, dtype=torch.uint8) tensor(0, dtype=torch.uint8)
tensor(0, dtype=torch.uint8) tensor(236, dtype=torch.uint8) after transform
tensor(0, dtype=torch.uint8)
tensor(253, dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor(253, dtype=torch.uint8) tensor(0,

 tensor(0, dtype=torch.uint8)after transformtensor(12, dtype=torch.uint8) tensor(232, dtype=torch.uint8)
  tensor(243, dtype=torch.uint8)tensor(203, dtype=torch.uint8)tensor(4, dtype=torch.uint8)tensor(255, dtype=torch.uint8)  after transform tensor(232, dtype=torch.uint8) 
tensor(0, dtype=torch.uint8) tensor(0, dtype=torch.uint8)before 255, 0tensor(12, dtype=torch.uint8)tensor(255, dtype=torch.uint8) 
tensor(4, dtype=torch.uint8)
 

tensor(243, dtype=torch.uint8)tensor(0, dtype=torch.uint8)  tensor(0, dtype=torch.uint8)tensor(255, dtype=torch.uint8)
 tensor(0, dtype=torch.uint8)
after transform
tensor(255, dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor(255, dtype=torch.uint8) before 255, 0before 255, 0tensor(0, dtype=torch.uint8)


before 255, 0
before 255, 0before 255, 0

after transform
tensor(200, dtype=torch.uint8) before 255, 0tensor(0, dtype=torch.uint8)
 after transform
tensor(200, dtype=torch.uint8)tensor(255, dtype=torch.uint8) after transform tensor(0, dtype=torch.ui

  tensor(255, dtype=torch.uint8) after transform tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)


  before 255, 0tensor(255, dtype=torch.uint8)tensor(252, dtype=torch.uint8)tensor(255, dtype=torch.uint8)
   tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)
 
before 255, 0tensor(255, dtype=torch.uint8)
 tensor(0, dtype=torch.uint8)after transform

tensor(249, dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor(249, dtype=torch.uint8) after transformtensor(0, dtype=torch.uint8)

after transformtensor(212, dtype=torch.uint8)
 tensor(0, dtype=torch.uint8) tensor(255, dtype=torch.uint8)tensor(212, dtype=torch.uint8)  tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8) 
tensor(255, dtype=torch.uint8) tensor(0, dtype=torch.uint8)
before 255, 0
before 255, 0
before 255, 0
after transform
tensor(226, dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor(226, dtype=torch.uint8) tensor(0,

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 tensor(255, dtype=torch.uint8) tensor(0, dtype=torch.uint8)
before 255, 0
before 255, 0
before 255, 0
after transformafter transform

tensor(246, dtype=torch.uint8) tensor(246, dtype=torch.uint8)tensor(0, dtype=torch.uint8)before 255, 0  tensor(246, dtype=torch.uint8)
 tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)
 tensor(246, dtype=torch.uint8) tensor(0, dtype=torch.uint8)
after transform
tensor(224, dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor(224, dtype=torch.uint8) tensor(0, dtype=torch.uint8)
before 255, 0after transform

tensor(242, dtype=torch.uint8) tensor(0, dtype=torch.uint8) before 255, 0tensor(242, dtype=torch.uint8) 
tensor(0, dtype=torch.uint8)
after transform
tensor(233, dtype=torch.uint8) before 255, 0tensor(0, dtype=torch.uint8)
 tensor(233, dtype=torch.uint8) tensor(0, dtype=torch.uint8)
before 255, 0
after transform
tensor(242, dtype=torch.uint8)after transform 
tensor(0, dtype=torch.uint8) tensor(242, dtype=torch.uint8)tensor(236, dtype=torch.u

# And finally, inference!
note: calculating the KNN for the train embeddings takes a while

In [187]:
K = 2 # choose a K for the KNN algorithm, paper says 2 performs slightly better however not much variability in performance
loaders = {'deepfake' : dpfk_test_loader, 'f2f' : f2f_test_loader, 'nt' : nt_test_loader}
models = {'original_augs' : original_aug_model, 'new_augs' : new_aug_model,'new_augs_no_pre' : new_aug_no_pre_model,
          'new_augs_no_pre2' : new_aug_no_pre_model2, 'resnet':resnet_model}
embeds_dict = {'original_augs' : original_aug_train_embeds, 
               'new_augs': new_aug_train_embeds, 
               'new_augs_no_pre' : new_aug_no_pre_train_embeds,
               'new_augs_no_pre2' : new_aug_no_pre_train_embeds2,
               'resnet':resnet_train_embeds}
results_df = pd.DataFrame(columns=['model_name', 'deepfake_method',
                                   'classification_accuracy', 'knn_accuracy',
                                  'knn_dist_threshold','train_size','test_size'])
for model_name, model in models.items():
    train_embeds = embeds_dict[model_name]
    train_embeds = train_embeds[train_targets==1] #we use only "normal" aka real samples for the train set
    print(f'Calculating KNNs on {model_name} model embeddings')
    # get the sklearn Nearest Neighbor calculator and the KNN distances of the train set, the latter will be used
    # for thresholding the test set to determine "anomalies" aka fake samples:
    train_nn, train_knns = get_train_knns(train_embeds.cpu(), k=K)  
    for loader_name, loader in loaders.items():
        print(f'Getting test embeddings')
        undiv_embeds, test_embeds,test_targets = get_embeds(model,loader,div=True)
        print(f'Performing linear eval on {loader_name}')
        if model_name=='new_augs_no_pre2':
            c_acc,c_pred,c_tar = eval_model_classification(model,undiv_embeds,test_targets)
        else:
            c_acc,c_pred,c_tar = eval_model_classification(model,test_embeds,test_targets)
        print(f'Calculating {loader_name} KNNs')
        k_acc,k_pred,k_tar,k_thresh = eval_model_knns(test_embeds, test_targets, train_nn, train_knns,k=K)
        print(f'model: {model_name}, manipulation: {loader_name}, linear eval accuracy: {c_acc:.2f}, knn accuracy: {k_acc:.2f}')
        results_df = results_df.append({'model_name' : model_name, 
                                        'deepfake_method' : loader_name,
                                        'classification_accuracy' : c_acc, 
                                        'knn_accuracy' : k_acc,
                                       'knn_dist_threshold':k_thresh,
                                       'train_size': len(train_knns),
                                       'test_size': len(loader.dataset)}, ignore_index=True)
        
results_path = '/media/shirbar/DATA/Deep_Learning/Project/knn_linear_eval_comparison_test_divided.csv'
results_df.to_csv(results_path)
print(f'Saved results to file at {results_path}')

Calculating KNNs on original_augs model embeddings


  0%|          | 0/375 [00:00<?, ?it/s]

Getting test embeddings


  0%|          | 1/375 [00:00<03:22,  1.85it/s]

before: 255, 0
after: 1.0, 0.0


...

model: resnet, manipulation: f2f, linear eval accuracy: 51.83, knn accuracy: 53.95
Getting test embeddings


  0%|          | 1/375 [00:00<02:44,  2.27it/s]

before: 255, 0
after: 1.0, 0.0


100%|██████████| 375/375 [02:59<00:00,  2.09it/s]


Performing linear eval on nt

total real: tensor(3436, device='cuda:0')  total fake: tensor(2564, device='cuda:0')

PREDS:
 tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0') 
TARGETS:
 tensor([1, 0, 0,  ..., 0, 1, 0], device='cuda:0', dtype=torch.int8)
total correct: tensor(3580, device='cuda:0')  out of: 6000

ACC: 59.666664123535156
Calculating nt KNNs
model: resnet, manipulation: nt, linear eval accuracy: 59.67, knn accuracy: 55.78
Saved results to file at /media/shirbar/DATA/Deep_Learning/Project/knn_linear_eval_comparison_test_divided.csv


In [190]:
results_df.knn_accuracy = results_df.knn_accuracy.apply(lambda x: x.item())
results_df

,model_name,deepfake_method,classification_accuracy,knn_accuracy,knn_dist_threshold,train_size,test_size
0,original_augs,deepfake,"tensor(99.2667, device='cuda:0')",52.016670,7.297311,28000,6000
1,original_augs,f2f,"tensor(50.8833, device='cuda:0')",49.750000,7.297311,28000,6000
2,original_augs,nt,"tensor(61.9667, device='cuda:0')",43.849998,7.297311,28000,6000
3,new_augs,deepfake,"tensor(84.7667, device='cuda:0')",49.833332,3.157340,28000,6000
4,new_augs,f2f,"tensor(62.6167, device='cuda:0')",49.833332,3.157340,28000,6000
5,new_augs,nt,"tensor(69.3167, device='cuda:0')",43.333332,3.157340,28000,6000
6,new_augs_no_pre,deepfake,"tensor(98.1000, device='cuda:0')",51.533337,7.999893,28000,6000
7,new_augs_no_pre,f2f,"tensor(65.3000, device='cuda:0')",51.116669,7.999893,28000,6000
8,new_augs_no_pre,nt,"tensor(71.6500, device='cuda:0')",45.450001,7.999893,28000,6000
9,new_augs_no_pre2,deepfake,"tensor(98.5333, device='cuda:0')",51.483334,7.999893,28000,6000


In [12]:
### Check our winning model - new_augs_no_pre on allllll the test set, start by loading the test sets:
TEST_SIZE = -1
TRANSFORMS = original_aug_model.trainset.dataset.dataset.transform #all classifiers have the same transforms
split='test'

dpfk_test_loader = load_ff_ds_test(wanted_ds_size=TEST_SIZE,
                    batch_size=16,
                    load_deepfakes=True,
                    load_face2face=False,
                    load_neural_textures=False,
                    masking_transforms=False,
                    transform=TRANSFORMS,
                    split=split) 
f2f_test_loader = load_ff_ds_test(wanted_ds_size=TEST_SIZE,
                    batch_size=16,
                    load_deepfakes=False,
                    load_face2face=True,
                    load_neural_textures=False,
                    masking_transforms=False,
                    transform=TRANSFORMS,
                    split=split)
nt_test_loader = load_ff_ds_test(wanted_ds_size=TEST_SIZE,
                    batch_size=16,
                    load_deepfakes=False,
                    load_face2face=False,
                    load_neural_textures=True,
                    masking_transforms=False,
                    transform=TRANSFORMS,
                    split=split)


loading real images...
real images loaded!
loading deepfakes...


KeyboardInterrupt: 

In [20]:
# Now let's do some inferring:
K = 2
all_test_results_df = pd.DataFrame(columns=['model_name','deepfake_method',
                                   'classification_accuracy', 'knn_accuracy',
                                  'knn_dist_threshold','train_size','test_size'])
loaders = {'deepfake' : dpfk_test_loader, 'f2f' : f2f_test_loader, 'nt' : nt_test_loader}

model = new_aug_no_pre_model
model_name = 'new_augs_no_pre'

train_embeds = new_aug_no_pre_train_embeds
train_embeds = train_embeds[train_targets==1] # only real data in the trainset
print(f'Calculating KNNs on {model_name} model embeddings')
train_nn, train_knns = get_train_knns(train_embeds, k=K)  

for loader_name, loader in loaders.items():
    print(f'Performing linear eval on {loader_name}')
    test_embeds,test_targets = get_embeds(model,loader)
    c_acc,c_pred,c_tar = eval_model_classification(test_embeds, test_targets, no_pre = model_name=='new_augs_no_pre')
    print(f'Calculating {loader_name} KNNs')
    k_acc,k_pred,k_tar,k_thresh = eval_model_knns(model,loader, train_nn, train_knns,k=K)
    print(f'manipulation: {loader_name}, linear eval accuracy: {c_acc:.2f}, knn accuracy: {k_acc:.2f}')
    all_test_results_df = all_test_results_df.append({'model_name' : model_name,
                                        'deepfake_method' : loader_name,
                                        'classification_accuracy' : c_acc, 
                                        'knn_accuracy' : k_acc,
                                       'knn_dist_threshold':k_thresh,
                                       'train_size': len(train_knns),
                                       'test_size': len(loader.dataset)}, ignore_index=True)
    
results_path = '/media/shirbar/DATA/Deep_Learning/Project/new_aug_no_pretrained_all_test_results.csv'
results_df.to_csv(results_path)
print(f'Saved results to file at {results_path}')

Calculating KNNs on new_augs_no_pre model embeddings


  0%|          | 0/9221 [00:00<?, ?it/s]

Performing linear eval on deepfake


 12%|█▏        | 1123/9221 [10:29<2:43:36,  1.21s/it]

invalid mask
remasking: /media/shirbar/My Passport/FaceForensics/split_ds/test/original_sequences/youtube/c23/images/257  to dest: /media/shirbar/My Passport/FaceForensics/split_ds/test/original_sequences/youtube/masks


...

invalid mask
remasking: /media/shirbar/My Passport/FaceForensics/split_ds/test/manipulated_sequences/Deepfakes/c23/images/024_073  to dest: /media/shirbar/My Passport/FaceForensics/split_ds/test/manipulated_sequences/Deepfakes/masks/images
no face was detected at image: /media/shirbar/My Passport/FaceForensics/split_ds/test/manipulated_sequences/Deepfakes/c23/images/024_073/0225.png
no face was detected at image: /media/shirbar/My Passport/FaceForensics/split_ds/test/manipulated_sequences/Deepfakes/c23/images/024_073/0226.png
invalid mask
remasking: /media/shirbar/My Passport/FaceForensics/split_ds/test/manipulated_sequences/Deepfakes/c23/images/024_073  to dest: /media/shirbar/My Passport/FaceForensics/split_ds/test/manipulated_sequences/Deepfakes/masks/images
no face was detected at image: /media/shirbar/My Passport/FaceForensics/split_ds/test/manipulated_sequences/Deepfakes/c23/images/024_073/0225.png
no face was detected at image: /media/shirbar/My Passport/FaceForensics/split_ds/t

  0%|          | 0/9221 [00:00<?, ?it/s]

manipulation: deepfake, linear eval accuracy: 98.66, knn accuracy: 79.56
Performing linear eval on f2f


 69%|██████▊   | 6320/9221 [3:20:03<1:31:49,  1.90s/it] 


KeyboardInterrupt: 

In [59]:
train, div_train, tr = get_embeds(new_aug_no_pre_model, train_loader,div=True)


  0%|          | 3/875 [00:01<18:32,  1.28s/it]

before: 255, 0
after: 1.0, 0.0


100%|██████████| 875/875 [01:39<00:00,  8.75it/s]


In [60]:
train_nn_div, train_knns_div = get_train_knns(div_train.cpu(), k=2,p=2)  


In [ ]:
train_nn, train_knns = get_train_knns(train.cpu(), k=2,p=2)  


In [ ]:
train_nn.effective_metric_ 

In [56]:
train_nn_div.kneighbors(div_tests[trs==1]=)

(array([[479.89738835, 507.69401658]]), array([[25083, 13925]]))

In [25]:
test, div_tests, trs = get_embeds(new_aug_no_pre_model, dpfk_test_loader,div=True)

  0%|          | 1/375 [00:00<03:38,  1.71it/s]

before: 255, 0
after: 1.0, 0.0


100%|██████████| 375/375 [02:57<00:00,  2.11it/s]


In [38]:
eval_model_knns(test.cpu(), trs, train_nn, train_knns,k=2)

(tensor(80.4833),
 tensor([1., 0., 0.,  ..., 0., 0., 0.]),
 tensor([1, 0, 0,  ..., 0, 1, 1], dtype=torch.int8),
 65204.95803739357)

In [26]:
test_knns_div,inds = train_nn_div.kneighbors(div_tests.cpu())

In [27]:
test_knns_div.min()

153.04249597582384

In [33]:
((test_knns_div.mean(axis=1)<np.quantile(train_knns_div,0.99)).sum())/(trs.sum())

tensor(0.3311)

In [53]:

train_nn_div, train_knns_div = get_train_knns(div_ts.cpu(), k=K)  


[autoreload of dfd_utils.FaceForensicsDataset failed: Traceback (most recent call last):
  File "/home/shirbar/anaconda3/envs/simclr_pytorch/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/shirbar/anaconda3/envs/simclr_pytorch/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 410, in superreload
    update_generic(old_obj, new_obj)
  File "/home/shirbar/anaconda3/envs/simclr_pytorch/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/shirbar/anaconda3/envs/simclr_pytorch/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 302, in update_class
    if update_generic(old_obj, new_obj): continue
  File "/home/shirbar/anaconda3/envs/simclr_pytorch/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 347, in update_generic
    update(a, b)
  File "/home/shirbar/anaconda3/envs/simclr_

255255255255  00  

00
255
255  00

255 0
tensor(255, dtype=torch.uint8) tensor(231, dtype=torch.uint8)tensor(255, dtype=torch.uint8) tensor(254, dtype=torch.uint8)tensor(241, dtype=torch.uint8)tensor(0, dtype=torch.uint8) tensor(9, dtype=torch.uint8)    tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)tensor(255, dtype=torch.uint8)tensor(231, dtype=torch.uint8)   tensor(246, dtype=torch.uint8) tensor(255, dtype=torch.uint8)tensor(254, dtype=torch.uint8) tensor(0, dtype=torch.uint8)tensor(9, dtype=torch.uint8)  

 tensor(0, dtype=torch.uint8)tensor(241, dtype=torch.uint8)tensor(0, dtype=torch.uint8)
  tensor(0, dtype=torch.uint8)tensor(246, dtype=torch.uint8)
tensor(0, dtype=torch.uint8) tensor(231, dtype=torch.uint8)
tensor(0, dtype=torch.uint8) tensor(0, dtype=torch.uint8)
 255tensor(231, dtype=torch.uint8)  tensor(0, dtype=torch.uint8)0

255 0
199 24
255 0
tensor(214, dtype=torch.uint8)tensor(225, dtype=torch.uint8)  tensor(0, dtype=torch.uint8)ten

 tensor(255, dtype=torch.uint8) tensor(246, dtype=torch.uint8)tensor(0, dtype=torch.uint8)tensor(255, dtype=torch.uint8) 
 tensor(14, dtype=torch.uint8)tensor(215, dtype=torch.uint8)tensor(0, dtype=torch.uint8)  tensor(246, dtype=torch.uint8) 255tensor(255, dtype=torch.uint8) tensor(218, dtype=torch.uint8)  tensor(14, dtype=torch.uint8)tensor(0, dtype=torch.uint8) 255
0 tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)
tensor(215, dtype=torch.uint8)   
tensor(218, dtype=torch.uint8)0tensor(0, dtype=torch.uint8)
 
tensor(0, dtype=torch.uint8)
255 0
255 0tensor(155, dtype=torch.uint8)
 tensor(0, dtype=torch.uint8)tensor(221, dtype=torch.uint8)  tensor(155, dtype=torch.uint8)tensor(0, dtype=torch.uint8)  255tensor(221, dtype=torch.uint8) tensor(0, dtype=torch.uint8)0 

tensor(0, dtype=torch.uint8)
255255 tensor(251, dtype=torch.uint8)tensor(230, dtype=torch.uint8) 00
  
tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)  255tensor(230, dtype=torch.uint8) tensor(251, dtype=tor

tensor(255, dtype=torch.uint8) tensor(0, dtype=torch.uint8)
255 tensor(255, dtype=torch.uint8)0 255
tensor(0, dtype=torch.uint8) 2552550   0tensor(255, dtype=torch.uint8)
0 

tensor(0, dtype=torch.uint8)
255255255   000


tensor(231, dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor(231, dtype=torch.uint8)tensor(218, dtype=torch.uint8)  tensor(0, dtype=torch.uint8)tensor(255, dtype=torch.uint8)tensor(0, dtype=torch.uint8)
  tensor(218, dtype=torch.uint8)tensor(0, dtype=torch.uint8)  tensor(0, dtype=torch.uint8)tensor(255, dtype=torch.uint8)
 tensor(0, dtype=torch.uint8)tensor(239, dtype=torch.uint8)
 tensor(251, dtype=torch.uint8)tensor(255, dtype=torch.uint8)tensor(0, dtype=torch.uint8)   tensor(0, dtype=torch.uint8)tensor(239, dtype=torch.uint8)tensor(16, dtype=torch.uint8)   tensor(251, dtype=torch.uint8)tensor(0, dtype=torch.uint8)tensor(255, dtype=torch.uint8)
  tensor(0, dtype=torch.uint8)tensor(193, dtype=torch.uint8)tensor(16, dtype=torch.uint8)
 
tensor(0, dtype=torch.uin

tensor(255, dtype=torch.uint8)tensor(255, dtype=torch.uint8)  tensor(218, dtype=torch.uint8)tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8)   tensor(255, dtype=torch.uint8)tensor(0, dtype=torch.uint8)tensor(255, dtype=torch.uint8)   tensor(0, dtype=torch.uint8)tensor(218, dtype=torch.uint8)tensor(0, dtype=torch.uint8)
 
tensor(0, dtype=torch.uint8)tensor(210, dtype=torch.uint8)
 tensor(0, dtype=torch.uint8) tensor(210, dtype=torch.uint8) tensor(0, dtype=torch.uint8)
tensor(209, dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor(255, dtype=torch.uint8)tensor(209, dtype=torch.uint8)255   tensor(0, dtype=torch.uint8)0 tensor(0, dtype=torch.uint8)
tensor(255, dtype=torch.uint8)
 tensor(0, dtype=torch.uint8)255
 0
255 0
tensor(255, dtype=torch.uint8) tensor(6, dtype=torch.uint8) tensor(255, dtype=torch.uint8) tensor(6, dtype=torch.uint8)tensor(255, dtype=torch.uint8)
 tensor(0, dtype=torch.uint8)tensor(255, dtype=torch.uint8)  tensor(255, dtype=torch.uint8)255tensor(0, dtype=to

...

 tensor(255, dtype=torch.uint8)255  tensor(0, dtype=torch.uint8)0

255 0
255255  0tensor(255, dtype=torch.uint8)0 

tensor(0, dtype=torch.uint8)255  tensor(255, dtype=torch.uint8)0 tensor(0, dtype=torch.uint8)

tensor(248, dtype=torch.uint8) 255tensor(0, dtype=torch.uint8)  0tensor(248, dtype=torch.uint8)
 tensor(0, dtype=torch.uint8)
255 0
tensor(255, dtype=torch.uint8)255  tensor(255, dtype=torch.uint8)0
tensor(0, dtype=torch.uint8)  tensor(0, dtype=torch.uint8) tensor(255, dtype=torch.uint8) tensor(255, dtype=torch.uint8)tensor(0, dtype=torch.uint8)tensor(255, dtype=torch.uint8) 
 tensor(255, dtype=torch.uint8)tensor(0, dtype=torch.uint8) tensor(0, dtype=torch.uint8)
tensor(0, dtype=torch.uint8) tensor(248, dtype=torch.uint8)  tensor(255, dtype=torch.uint8)tensor(255, dtype=torch.uint8)tensor(0, dtype=torch.uint8)  tensor(0, dtype=torch.uint8)tensor(0, dtype=torch.uint8) 

tensor(248, dtype=torch.uint8) tensor(0, dtype=torch.uint8)
tensor(253, dtype=torch.uint8) tensor(3, dtype=torc

 19%|█▊        | 164/875 [00:37<03:04,  3.86it/s]

tensor(229, dtype=torch.uint8) tensor(255, dtype=torch.uint8)tensor(0, dtype=torch.uint8) tensor(0, dtype=torch.uint8)255   0tensor(255, dtype=torch.uint8)tensor(229, dtype=torch.uint8)
  tensor(0, dtype=torch.uint8)
tensor(0, dtype=torch.uint8)
255 0255
 0
tensor(244, dtype=torch.uint8) tensor(0, dtype=torch.uint8) 255tensor(244, dtype=torch.uint8)  255tensor(0, dtype=torch.uint8)0 
0

tensor(220, dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor(220, dtype=torch.uint8) tensor(0, dtype=torch.uint8)tensor(255, dtype=torch.uint8)
 tensor(0, dtype=torch.uint8) 255tensor(255, dtype=torch.uint8)255   tensor(0, dtype=torch.uint8)00


tensor(255, dtype=torch.uint8) tensor(0, dtype=torch.uint8)tensor(253, dtype=torch.uint8)  tensor(8, dtype=torch.uint8) tensor(255, dtype=torch.uint8)tensor(253, dtype=torch.uint8)  tensor(8, dtype=torch.uint8)tensor(0, dtype=torch.uint8)

tensor(255, dtype=torch.uint8) tensor(0, dtype=torch.uint8) tensor(255, dtype=torch.uint8) 255tensor(0, dtype=torch.ui

 19%|█▉        | 165/875 [00:38<02:43,  4.33it/s]


KeyboardInterrupt: 

In [34]:
pred = model(tm.cuda())
pred = F.softmax(pred,dim=1)
pred_class = torch.argmax(pred, dim=1)
(pred_class==tr.cuda()).sum()
(pred_class==tr.cuda()).sum()/pred_class.shape[0]

tensor(0.9850, device='cuda:0')

In [51]:
eval_model_knns(div_tm.cpu(), tr, train_nn_div, train_knns_div,k=2)

(tensor(51.4833),
 tensor([0., 0., 0.,  ..., 0., 0., 0.]),
 tensor([1, 0, 0,  ..., 0, 1, 1], dtype=torch.int8),
 8.006193161010742)

In [16]:
dist = DistanceMetric.get_metric('euclidean')
train_dists = dist.pairwise(train_embeds[0:10].cpu())

In [26]:
test_embeds,test_targets = get_embeds(new_aug_model,dpfk_test_loader)

100%|██████████| 375/375 [03:00<00:00,  2.08it/s]


In [44]:
train_knns = np.take_along_axis(train_dists,
                                    np.argpartition(train_dists + np.diag(np.ones(train_dists.shape[0]) * 10000),
                                        3, axis=1)[:, :3], axis=1)
train_knns.sort(axis=1)

In [34]:
nns, train_knns = get_train_knns(train_embeds, k=2)

In [35]:
train_knns

array([[19.78044  , 26.767769 ],
       [39.74767  , 42.909885 ],
       [22.66722  , 24.380262 ],
       ...,
       [12.900446 , 16.08406  ],
       [ 2.1854553, 15.676921 ],
       [20.034128 , 26.160309 ]], dtype=float32)

In [26]:
t= np.argpartition(train_dists + np.diag(np.ones(train_dists.shape[0]) * 10),3,axis=1)

In [30]:
nbrs = NearestNeighbors(n_neighbors=3, algorithm='kd_tree').fit(train_embeds[0:10])

In [31]:
nns.kneighbors_graph(train_embeds[0:10]).toarray()

array([[1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 1.]])

In [32]:
nbrs.n_neighbors = 3
distances, indices = nbrs.kneighbors(train_embeds[0:10])
print(distances, indices)

[[  0.         191.73926375 195.17021662]
 [  0.         211.06366218 392.41110453]
 [  0.         211.06366218 243.78845252]
 [  0.          54.89667625 106.94911497]
 [  0.         181.26613846 211.67355839]
 [  0.          93.02510261  94.55981052]
 [  0.          58.76523835 138.24071196]
 [  0.          54.89667625  93.02510261]
 [  0.          58.76523835 132.10754925]
 [  0.          94.55981052 106.82116115]] [[0 6 8]
 [1 2 4]
 [2 1 4]
 [3 7 5]
 [4 5 7]
 [5 7 9]
 [6 8 9]
 [7 3 5]
 [8 6 3]
 [9 5 7]]


In [80]:
len(dpfk_test_loader.dataset)

6000

In [62]:
nbrs.kneighbors(test_embeds[0:2])

(array([[2887.82278236, 5358.12888873, 5881.88308146],
        [ 787.17023935,  949.29831289, 1950.8808401 ]]),
 array([[5, 7, 4],
        [9, 6, 3]]))

In [74]:
nbrs.n_neighbors = 3

In [31]:
(get_test_knns(nbrs, test_embeds,k=3).mean(axis=1)<np.quantile(train_knns,0.95))

0

In [73]:
nbrs.n_neighbors

4

100%|██████████| 375/375 [03:00<00:00,  2.08it/s]


(tensor(53.4000),
 tensor([0., 1., 1.,  ..., 1., 0., 1.]),
 tensor([0, 0, 0,  ..., 1, 1, 1], dtype=torch.int8))